# Objectives {.unnumbered}

1. Use **PySpark** to process the Lightcast dataset.
2. Engineer features from structured columns for salary prediction.
3. Train **Linear Regression model**.
4. Evaluate models using **RMSE** and **R²**.
5. Visualize predictions using diagnostic plots.
6. Push work to GitHub and submit the repository link.

# Setup {.unnumbered}

The instruction below provides you with general keywords for columns used in the lightcast file. See the data schema generated after the load dataset code above to use proper column name. For visualizations, tables, or summaries, please **customize colors, fonts, and styles** as appropriate to avoid a **2.5-point deduction**. Also, **provide a two-sentence explanation** describing key insights drawn from each section's code and outputs. 

1. Follow the steps below as necessary, use your best judgement in importing/installing/creating/saving files as needed.
2. Create a new Jupyter Notebook in your `ad688-sp25-lab08` directory named `lab08_yourname.ipynb`, if the file exists make sure to change the name.
3. Use your **EC2 instance** for this lab.
4. Ensure the `lightcast_data.csv` file is available on the EC2 instance. if not then **Download the dataset**
5. **Add the dataset to `.gitignore`** to avoid pushing large files to GitHub. Open your `.gitignore` file and add:
6. Make sure to create a virtual environment and install the required Python libraries if needed, don't forget to activate it:
7. Install the required Python libraries if needed, you can also use the given requirement file to install the packages to the virtual environment:

```bash
python3 -m venv .venv
source .venv/bin/activate
gdown https://drive.google.com/uc?id=1V2GCHGt2dkFGqVBeoUFckU4IhUgk4ocQ
echo "lightcast_job_postings.csv" >> .gitignore
pip install -r requirements.txt
```


# Load the Dataset
1. **Load the Raw Dataset**:
   - Use Pyspark to the `lightcast_data.csv` file into a DataFrame:
   - You can reuse the previous code. 
   - [Copying code from your friend constitutes plagiarism. DO NOT DO THIS]{.uured-bold}.

In [1]:
#| eval: true
#| echo: true
from pyspark.sql import SparkSession
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 20)

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/14 17:09:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

# Feature Engineering

Feature Engineering is a crucial step in preparing your data for machine learning. In this lab, we will focus on the following tasks:

1. **Drop rows with missing values** in the target variable and key features.
2. By now you are already familiar with the code and the data. Based on your understanding please choose any 3 (my code output has 10) variables as:
   1. two continuous variables (use your best judgment!)
   2. one categorical.
   3. Your dependent variable (y) is `SALARY`.
3. **Convert categorical variables** into numerical representations using **StringIndexer** and **OneHotEncoder**.
4. **Assemble features** into a single vector using **VectorAssembler**.
5. **Split the data** into training and testing sets.

In [30]:
#| eval: false
#| echo: true
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

df = df.dropna(subset=[
    "SALARY",
    "REMOTE_TYPE_NAME",
    "DURATION",
    "MIN_YEARS_EXPERIENCE",
])
# from previous labs, we know that average salary of remote/on site/hybrid are quite different
categorical_cols = ["MIN_EDULEVELS_NAME"]
# we only have two numerical variables: Duration and YOE
continuous_cols = ["DURATION", "MIN_YEARS_EXPERIENCE"]

indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_idx", handleInvalid='skip') for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=f"{col}_idx", outputCol=f"{col}_vec") for col in categorical_cols]

assembler = VectorAssembler(
    inputCols=continuous_cols + [f"{col}_vec" for col in categorical_cols],
    outputCol="features",
)

pipeline = Pipeline(stages=indexers + encoders + [assembler])
data = pipeline.fit(df).transform(df).select("SALARY", "features")
data.show(5,False)

+------+---------------------------+
|SALARY|features                   |
+------+---------------------------+
|192800|(7,[0,1,2],[55.0,6.0,1.0]) |
|125900|(7,[0,1,5],[18.0,12.0,1.0])|
|118560|(7,[0,1,3],[20.0,5.0,1.0]) |
|192800|(7,[0,1,2],[55.0,6.0,1.0]) |
|116500|(7,[0,1,5],[16.0,12.0,1.0])|
+------+---------------------------+
only showing top 5 rows



# Train/Test Split

- Perform a **random split** of the data into training and testing sets.
- Set a random seed for reproducibility.
- You can choose a number for splitting to your liking, justify your choice.

In [31]:
data.count()

14416

In [32]:
# After dropping NA, we have only 14k samples.
# The dataset is small, so I chooe to split by 70/30.
# The seed is set to 42 for reproducibility
train_data, test_data = data.randomSplit([0.7, 0.3], seed=42)
print((train_data.count(), len(train_data.columns)))
print((test_data.count(), len(test_data.columns)))

(10217, 2)


[Stage 56:>                                                         (0 + 1) / 1]

(4199, 2)


# Linear Regression

- Train a **Linear Regression** model using the training data. [You will run in to an important issue here. Please make an effort in figuring it by yourself. This is one of the most asked interview questions in CapitalOne's management recruiting program.]{.uured-bold}
- Evaluate the model on the test data.
- Print the coefficients, intercept, R², RMSE, and MAE.
- Use the `summary` object to extract the coefficients and their standard errors, t-values, and p-values.
- Create a DataFrame to display the coefficients, standard errors, t-values, p-values, and confidence intervals.
- Interpret the coefficients and their significance and explain the model performance metrics.

In [33]:
# train a linear regression model
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol="features", labelCol="SALARY", predictionCol="prediction")
lr_model = lr.fit(train_data)

25/04/14 17:35:38 WARN Instrumentation: [bac1477f] regParam is zero, which might cause numerical instability and overfitting.
                                                                                

In [34]:
# model summary
summary = lr_model.summary
pd.DataFrame(dict(
    r2 = [summary.r2],
    rmse = [summary.rootMeanSquaredError],
    mae = [summary.meanAbsoluteError],
))

,r2,rmse,mae
0,0.375663,33844.806259,26245.314927


Interpretaion of model summaries:

- R-square 0.376: only 37% variation in salries can be explained by the model
- RMSE and MAE: very large, the model cannot give precise predictions

We have only take 3 variables as input, many other variables may play have significant influence on salaries.

In [35]:
# evaluate on test data
from pyspark.ml.evaluation import RegressionEvaluator
predictions = lr_model.transform(test_data)

# supported metrics: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.RegressionEvaluator.html
test_eval = dict()
for metric in ["rmse", "r2", "mae"]:
    evaluator = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName=metric)
    value = evaluator.evaluate(predictions)
    test_eval[metric] = [value]

pd.DataFrame.from_dict(test_eval)

,rmse,r2,mae
0,33134.004218,0.369048,25758.992898


Interpreations of results on test set:

- RMSE/MAE/R-Square are similar to the result on training set
- our model can genearlize to unseen data
- it has a consistent performance on train/test set

However, as we said before, the model cannot give a precise prediction and can only explain for 37% of the variation in the salary

In [39]:
# show intercept and coefficients
pd.DataFrame({
    "Intercept and Coefficients": [lr_model.intercept] + list(lr_model.coefficients),
    "Std Error": summary.coefficientStandardErrors,
    "t-stat": summary.tValues,
    "P-Value": summary.pValues
})

,Intercept and Coefficients,Std Error,t-stat,P-Value
0,98812.681381,23.372127,-0.313938,7.535746e-01
1,-7.337397,106.549389,70.225843,0.000000e+00
2,7482.520653,8222.987333,-2.011999,4.424626e-02
3,-16544.642263,8257.008684,-1.785402,7.422586e-02
4,-14742.077469,8305.820642,-5.765106,8.397403e-09
5,-47883.936060,8337.317900,-5.931231,3.104303e-09
6,-49450.554328,8494.582812,3.285598,1.021094e-03
7,27909.782503,8239.300243,11.992849,0.000000e+00


Interpretations:

- Intercept: This is the base salary level when no information is provided. Small std-err and very low p-value. The intercept is good enough.
- coefficient for duration: Negatively correlated to the salary. This means jobs that high paying jobs are quickly filled on the job market. However, this factor is not very significant, one may day open reduces the salary prediction by only 7.3$
- coefficient for years of experience: positively related to the salary. This means the more experience one have, the higher salary they get.
- coefficient for remote: On-site jobs are likely to have higher salaries (feature 7).
- For REMOTE type features: the coefficient absolute values are very large, meaning that remote/hybrid/onsite greatly influence the salary.
- The standard error of one-hot vector features are very large, meaning that the coefficient are mot reliable.

In [10]:
# Use the summary object to extract the coefficients and their standard errors, t-values, and p-values.
# Create a DataFrame to display the coefficients, standard errors, t-values, p-values, and confidence intervals.



coef_df.index = ["Intercept"] + [f"Feature {i+1}" for i in range(len(lr_model.coefficients))]


coeff_data = zip(
    ["Intercept"] + feature_names,
    [lr_model.intercept] + list(lr_model.coefficients),
    summary.coefficientStandardErrors,
    summary.tValues,
    summary.pValues
)

import pandas as pd

coef_df = pd.DataFrame({
    "Estimate": [lr_model.intercept] + list(lr_model.coefficients),
    "Std Error": lr_summary.coefficientStandardErrors,
    "t-stat": lr_summary.tValues,
    "P-Value": lr_summary.pValues
})

coef_df.index = ["Intercept"] + [f"Feature {i+1}" for i in range(len(lr_model.coefficients))]

coef_df.head()


# Summary stats
print("\n--- Regression Summary ---")
print("Coefficient Standard Errors:", [f"{val:.4f}" for val in summary.coefficientStandardErrors])
print("T Values:", [f"{val:.4f}" for val in summary.tValues])
print("P Values:", [f"{val:.4f}" for val in summary.pValues])

# print(f"\nDispersion: {summary.dispersion:.4f}")
# print(f"Null Deviance: {summary.nullDeviance:.4f}")
# print(f"Residual DF Null: {summary.residualDegreeOfFreedomNull}")
# print(f"Deviance: {summary.deviance:.4f}")
# print(f"Residual DF: {summary.residualDegreeOfFreedom}")
# print(f"AIC: {summary.aic:.4f}")

# 1. Pull feature names directly from Java backend
# feature_names = summary._call_java("featureNames")

# 2. Construct full table including intercept
features = ["Intercept"] + feature_names
coefs = [lr_model.intercept] + list(lr_model.coefficients)
se = list(summary.coefficientStandardErrors)
tvals = list(summary.tValues)
pvals = list(summary.pValues)

print(

print("---This is Diagnostic check, No need to print it in the final doc---")
print("Length of features:", len(features))
print("Length of coefs:", len(coefs))
print("Length of se:", len(se))
print("Length of tvals:", len(tvals))
print("Length of pvals:", len(pvals))

SyntaxError: invalid syntax (3268840423.py, line 49)

## Generalized Linear Regression Summary
The summary of the Generalized Linear Regression model provides important insights into the model's performance and the significance of each feature. The coefficients indicate the relationship between each feature and the target variable (salary), while the standard errors, t-values, and p-values help assess the reliability of these estimates.

- Please interpret them in the context of your data and model. 
- Feature Names are purposefully not printed in the output. You can use the `features` variable to print them out.

In [ ]:
#| eval: true
#| echo: false
import pandas as pd
from tabulate import tabulate
import pandas as pd


coef_table = pd.DataFrame({
    # "Feature": features,
    "Estimate": ,
    "Std Error": ,
    "t-stat": ,
    "P-Value": 
})

# 4. Optional pretty print
print(tabulate(coef_table, headers="keys", tablefmt="pretty"))

# 5. Save for report
# coef_table.to_csv("_output/glr_summary_pretty.csv", index=False)


# Diagnostic Plot

Diagnostic plots are essential for evaluating the performance of regression models. In this section, we will create several diagnostic plots to assess the linear regression model's assumptions and performance. There are four (2*2 grid) main plots we will create, you can use `seaborn` or `matplotlib` for this:

1. **Predicted vs Actual Plot**
2. **Residuals vs Predicted Plot**
3. **Histogram of Residuals**
4. **QQ Plot of Residuals**


In [ ]:
#| eval: true
#| echo: false
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# Load predictions from GLR model
df_pred = summary.predictions.select()

# Compute residuals
df_pred["residuals"] = 
df_pred["fitted"] = 

# Standardized residuals
res_mean = 
res_std = 
df_pred["std_residuals"] = 

# Square root of standardized residuals (for Scale-Location)
df_pred["sqrt_std_resid"] = 


plt.figure(figsize=(16, 12))
sns.set_theme(style="whitegrid")

# Plot 1: Residuals vs Fitted
plt.subplot(2, 2, 1)


# Plot 2: Normal Q-Q
plt.subplot(2, 2, 2)


# Plot 3: Scale-Location
plt.subplot(2, 2, 3)


# Plot 4: Residuals vs Leverage — Approximate
# Note: Leverage & Cook's Distance require X matrix; we approximate using fitted & residual
plt.subplot(2, 2, 4)


plt.tight_layout()
plt.savefig("_output/glr_diagnostic_classic.png")
plt.show()

# Evaluation

The evaluation of the model is crucial to understand its performance. In this section, we will calculate and visualize the following metrics:
1. **R² (Coefficient of Determination)**: Indicates how well the model explains the variance in the target variable.
2. **RMSE (Root Mean Squared Error)**: Measures the average magnitude of the errors between predicted and actual values.

In [ ]:
#| eval: true
#| echo: false
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, pow, sqrt, avg
import numpy as np

pred_glr = lr_model.transform(test_data)

# R²
r2_eval = 
r2 = 
# AIC from GLR summary
aic = 

# BIC calculation
n = 
k = 
rss = 
bic = 

# RMSE manually
residuals_df = 
rmse = 

## Model Evaluation Plot

- Display the predicted vs actual salary plot with a red line indicating the ideal fit (y=x).
- Use `seaborn` or `matplotlib` to create the plot.
- Customize the plot with appropriate titles, labels, and legends.
- Describe the plot in a few sentences, highlighting key insights and observations.

In [ ]:
#| eval: true
#| echo: false
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Convert GLR predictions to pandas
pandas_df = 

# Plot
plt.figure(figsize=(8, 6))


plt.title(f"Predicted vs Actual Salary (GLR Model)\n"
          f"RMSE = {rmse:.2f} | R² = {r2:.4f} | AIC = {aic:.2f} | BIC = {bic:.2f}", loc="left", fontsize=14, fontweight="bold")

plt.tight_layout()

# Save figure
plt.savefig("_output/glr_predicted_vs_actual.png", dpi=300)
plt.show()

# Submission {.unnumbered}
1. Save figures in the `_output/` folder.
2. Commit and push code and output files:
```bash
git add .
git commit -m "Add Lab 08 Salary Prediction models and output"
git push origin main
```
3. Submit your GitHub repository link.

# Resources {.unnumbered}
- [PySpark MLlib Docs](https://spark.apache.org/docs/latest/ml-guide.html)  
- [Seaborn Docs](https://seaborn.pydata.org/)  
- [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html)
